# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/10789/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,955,Laura,0.628929,-0.291060
1,1057,Norbert,-0.196602,0.249973
2,1025,George,-0.740080,-0.945661
3,951,Alice,0.298532,0.953015
4,1037,Sarah,-0.591305,0.849467


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999784e+02,3.434826e-04,-1.883365e-04
std,3.161074e+01,5.772180e-01,5.772312e-01
min,8.430000e+02,-9.999997e-01,-1.000000e+00
25%,9.790000e+02,-4.942074e-01,-4.913645e-01
50%,1.000000e+03,7.860631e-03,7.183494e-03
75%,1.022000e+03,5.059955e-01,5.047472e-01
max,1.173000e+03,9.999992e-01,9.999983e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
976,Patricia,0.6874577823361327
1051,Zelda,0.6779825957665729
966,Patricia,0.017463134418606474
1008,Jerry,0.6834765829026271
958,Patricia,0.4755173082144613
990,Bob,-0.5226595212421872
1016,Victor,0.2411242435691565
987,Hannah,0.5610249441714836
1041,Bob,-0.8707377220892094


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,955,Laura,0.628929,-0.291060
1,1057,Norbert,-0.196602,0.249973
2,1025,George,-0.740080,-0.945661
3,951,Alice,0.298532,0.953015
4,1037,Sarah,-0.591305,0.849467


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((subgraph_callable, (<function pandas_read_text at 0x7f39f4b448b0>, <function _make_parser_function.<locals>.parser_f at 0x7f39f6ca9af0>, (<function read_block_from_file at 0x7f39eeb00280>, <OpenFile '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 64000000, b'\n'), b'id,name,x,y\n', {}, {'id': dtype('int64'), 'name': dtype('O'), 'x': dtype('float64'), 'y': dtype('float64')}, ['id', 'name', 'x', 'y'], False, False, None), 'read-csv-f4077bb3ba6418015c145314fa9eada8'))
kwargs:    {}
Exception: ValueError('Length mismatch: Expected axis has 3 elements, new values have 4 elements')



ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,955,Laura,0.628929,-0.291060
1,1057,Norbert,-0.196602,0.249973
2,1025,George,-0.740080,-0.945661
3,951,Alice,0.298532,0.953015
4,1037,Sarah,-0.591305,0.849467


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,976,Patricia,0.687458,NaN
1,1051,Zelda,0.677983,NaN
2,966,Patricia,0.017463,NaN
3,1008,Jerry,0.683477,NaN
4,958,Patricia,0.475517,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-e97d3bec-55a6-4503-86d0-ea4e292531bd')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,955,Laura,0.628929,-0.291060
1,1057,Norbert,-0.196602,0.249973
2,1025,George,-0.740080,-0.945661
3,951,Alice,0.298532,0.953015
4,1037,Sarah,-0.591305,0.849467
...,...,...,...,...
86395,1016,Charlie,-0.996147,-0.827956
86396,986,Zelda,0.212479,-0.479421
86397,1026,Norbert,-0.806661,-0.645730
86398,1061,Laura,-0.912530,-0.510563


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-13.csv',
 'data/2000-01-28.csv',
 'data/2000-01-01.csv',
 'data/2000-01-08.csv',
 'data/2000-01-20.csv',
 'data/2000-01-12.csv',
 'data/2000-01-25.csv',
 'data/2000-01-30.csv',
 'data/2000-01-16.csv',
 'data/2000-01-11.csv',
 'data/2000-01-17.csv',
 'data/2000-01-27.csv',
 'data/2000-01-09.csv',
 'data/2000-01-18.csv',
 'data/2000-01-19.csv',
 'data/2000-01-26.csv',
 'data/2000-01-23.csv',
 'data/2000-01-05.csv',
 'data/2000-01-06.csv',
 'data/2000-01-03.csv',
 'data/2000-01-04.csv',
 'data/2000-01-02.csv',
 'data/2000-01-14.csv',
 'data/2000-01-24.csv',
 'data/2000-01-29.csv',
 'data/2000-01-21.csv',
 'data/2000-01-10.csv',
 'data/2000-01-15.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-c5ff327e-ab19-485d-a8db-4a603fcc800f'),
 Delayed('read_data-bded0dcc-adc0-425a-9b69-c7fe281cabcb'),
 Delayed('read_data-638f802d-21ec-41f2-babe-1b7b1a6ef736'),
 Delayed('read_data-397d7222-4c14-4be7-b33f-c2929a7f2e9e'),
 Delayed('read_data-3942ac49-9dcd-41c7-ae62-61c24150d384'),
 Delayed('read_data-cdbc7bfd-0cff-4a5a-bb25-462769506ea4'),
 Delayed('read_data-5bef5a78-734c-48d1-a358-44d9b7669676'),
 Delayed('read_data-b8bfdec1-98a5-494c-aedc-96dc8e86c864'),
 Delayed('read_data-be9a305a-b237-4d3a-9d5f-c203cbef22f0'),
 Delayed('read_data-3a8a0871-b8b4-42d8-95b5-bcca655cf90c'),
 Delayed('read_data-b6218ad8-e4d5-441d-bb69-ad8e53c0f7d1'),
 Delayed('read_data-c0f09c22-4cf4-49a0-a644-bb78c745e9c5'),
 Delayed('read_data-af8560fa-7ee1-4916-a5b7-d3890bac2a6b'),
 Delayed('read_data-7e647844-4a24-4008-bb25-f699ac0d11f9'),
 Delayed('read_data-b51ad915-4a71-44d9-865d-41a3651d3d14'),
 Delayed('read_data-854c951c-e84e-4851-a02c-5bf257f7891b'),
 Delayed('read_data-d282313c-47a6-412c-b

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_delayed). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1012,Yvonne,0.085485,-0.657034
1,973,Quinn,0.828762,0.061347
2,1003,Ingrid,0.035548,-0.773268
3,1012,Charlie,-0.237958,-0.693996
4,1045,Jerry,0.586854,-0.625294


In [26]:
df.describe().compute()

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999761e+02,2.358394e-04,-2.738392e-04
std,3.161207e+01,5.772135e-01,5.771832e-01
min,8.430000e+02,-9.999988e-01,-1.000000e+00
25%,9.790000e+02,-4.942074e-01,-4.913645e-01
50%,1.000000e+03,7.860631e-03,7.183494e-03
75%,1.022000e+03,5.051404e-01,5.042092e-01
max,1.173000e+03,9.999992e-01,9.999983e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.